# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [5]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"s

gpu_num = 1

os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_num)

In [6]:
!echo $CUDA_VISIBLE_DEVICES

1


In [9]:
### IJB-C Dataset
# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset
num_classes = 85742 
dataset_len = 5822653
batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
train_size = int(0.8 * dataset_len)
steps_per_epoch = train_size // batch_size
val_size = dataset_len - train_size
validation_steps = val_size // batch_size

w_decay=5e-4
epochs = 100

save_steps = 1000
steps = 1
is_ccrop=False
binary_img=True

is_Adam = False   # True
projection_head = False  # False
dgx = True

head_type = 'Svxhead' # ''ArcHead', CosHead', 'SphereHead', 'Svxhead'
# Backbones w/ pretrained weights:
#     MobileNet, MobileNetV2, InceptionResNetV2, InceptionV3, ResNet50, ResNet50V2, ResNet101V2, NASNetLarge, NASNetMobile, Xception, MobileNetV3Large, MobileNetV3Small, EfficientNetLite0~6, EfficientNetB0~7
# Backbones w/o pretrained weights:
#      MnasNetA1, MnasNetB1, MnasNetSmall 
backbone_type = 'ResNet50' 

if head_type == 'SphereHead':
    base_lr = 0.01 
    margin = 1.35
    logist_scale = 30.0 
elif head_type == 'CosHead':
    base_lr = 0.01 
    margin=0.35
    logist_scale=64
elif head_type == 'ArcHead':
    base_lr = 0.01 
#     base_lr = 0.1 
    margin=0.5
    logist_scale=64
elif head_type == 'Svxhead':
    base_lr = 0.01 
#     base_lr = 0.1 
    margin=0.5
    logist_scale=64
    t=0.2
else:
    base_lr = 0.01 # initially 0.01
    
print(head_type)
print(backbone_type)
print("projection head:", projection_head)
print("Adam:", is_Adam)
print("epoch:", epochs)
print("batch size:", batch_size)

Svxhead
ResNet50
projection head: False
Adam: False
epoch: 100
batch size: 128


In [10]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

# print("data: ", train_dataset)


strategy = tf.distribute.MirroredStrategy(devices=[f"/gpu:{gpu_num}"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             margin=margin, 
                             logist_scale=logist_scale,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True, 
                         projection_head=projection_head)
    model.summary()

    learning_rate = tf.constant(base_lr)
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate,
                                                                 decay_steps=60000, 
                                                   decay_rate=0.1,staircase=True)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=lr_schedule)
    else:
#         optimizer = tf.keras.optimizers.SGD(
#             learning_rate=learning_rate, momentum=0.9)
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=lr_schedule, momentum=0.9)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 4, 4, 2048)   23587712    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          16787968    resnet50[0][0]                   
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [ ]:
from pathlib import Path
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

callbacks = [mc_callback, tb_callback, early_stopping]
# callbacks = [mc_callback, tb_callback, early_stopping, lr_scheduler]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/100
 1000/36391 [..............................] - ETA: 56:29 - loss: 12.3212
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_Svxhead_check/SGD/e_1_l_12.23863697052002.ckpt
 2000/36391 [>.............................] - ETA: 55:07 - loss: 12.2225
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_Svxhead_check/SGD/e_1_l_12.008444786071777.ckpt
 3000/36391 [=>............................] - ETA: 53:57 - loss: 12.1140
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_Svxhead_check/SGD/e_1_l_11.789066314697266.ckpt
 4000/36391 [==>...........................] - ETA: 52:28 - loss: 12.0075
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_ResNet50_Svxhead_check/SGD/e_1_l_11.590241432189941.ckpt
 5000/36391 [===>..........................] - ETA: 50:31 - loss: 11.

### Resume training with latest checkpoint

In [6]:
from glob import glob
from pathlib import Path

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/{version}/*'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/{version}/*'
file_list = []
for files in glob(f'{base_dir}{save_name}'):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(f'{base_dir}{save_name}'):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

13
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/e_14_l_12.577579498291016.ckpt


In [7]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = f'{train_data_dir}ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img,
    is_ccrop=is_ccrop)

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if projection_head:
        save_name = f'ms1m_{backbone_type}_{head_type}_ProjectionHead_check/'
    else:
        save_name = f'ms1m_{backbone_type}_{head_type}_check/'
else:
    base_dir = "/hd/honghee/models/"
    save_name = f'ms1m_{backbone_type}_{head_type}_check/'

Path(f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}').mkdir(parents=True, exist_ok=True)

strategy = tf.distribute.MirroredStrategy(devices=["/gpu:{gpu_num}"])
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True,
                             projection_head=projection_head)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
check_dir = f'{base_dir}checkpoints/w_tfidentity/{save_name}{version}'
mc_callback = ModelCheckpoint(
            check_dir+'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=1, min_lr=0.001)
# callbacks = [mc_callback, tb_callback,reduce_lr]
callbacks = [mc_callback, tb_callback]

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-lite0 (Functional) (None, 4, 4, 1280)   3413024     input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          10493440    efficientnet-lite0[0][0]         
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [8]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 14/20
 1000/36391 [..............................] - ETA: 1:21:49 - loss: 12.4770
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_14_l_12.481388092041016.ckpt
 2000/36391 [>.............................] - ETA: 1:19:12 - loss: 12.4827
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_14_l_12.47399616241455.ckpt
 3000/36391 [=>............................] - ETA: 1:16:50 - loss: 12.4783
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_14_l_12.457012176513672.ckpt
 4000/36391 [==>...........................] - ETA: 1:14:34 - loss: 12.4737
Epoch 00014: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/che

24609/36391 [===================>..........] - ETA: 27:27 - loss: 12.2334
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_15_l_12.196710586547852.ckpt
25609/36391 [====================>.........] - ETA: 25:07 - loss: 12.2318
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_15_l_12.192112922668457.ckpt
26609/36391 [====================>.........] - ETA: 22:47 - loss: 12.2303
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_15_l_12.188915252685547.ckpt
27609/36391 [=====================>........] - ETA: 20:27 - loss: 12.2288
Epoch 00015: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidenti

18218/36391 [==============>...............] - ETA: 41:57 - loss: 12.0648
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_16_l_12.034358024597168.ckpt
19218/36391 [==============>...............] - ETA: 39:39 - loss: 12.0630
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_16_l_12.03027057647705.ckpt
20218/36391 [===============>..............] - ETA: 37:21 - loss: 12.0614
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_16_l_12.028227806091309.ckpt
21218/36391 [================>.............] - ETA: 35:03 - loss: 12.0598
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentit

11827/36391 [========>.....................] - ETA: 57:22 - loss: 11.9167
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_17_l_11.8868989944458.ckpt
12827/36391 [=========>....................] - ETA: 54:59 - loss: 11.9143
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_17_l_11.88347053527832.ckpt
13827/36391 [==========>...................] - ETA: 53:02 - loss: 11.9120
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_17_l_11.88149356842041.ckpt
14827/36391 [===========>..................] - ETA: 50:55 - loss: 11.9098
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/m

 5436/36391 [===>..........................] - ETA: 1:06:53 - loss: 11.7864
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_18_l_11.768781661987305.ckpt
 6436/36391 [====>.........................] - ETA: 1:04:42 - loss: 11.7834
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_18_l_11.765323638916016.ckpt
 7436/36391 [=====>........................] - ETA: 1:02:31 - loss: 11.7801
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_18_l_11.754659652709961.ckpt
 8436/36391 [=====>........................] - ETA: 1:00:22 - loss: 11.7772
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_

36391/36391 [==============================] - 4912s 135ms/step - loss: 11.7313 - val_loss: 11.8496
Epoch 19/20
   45/36391 [..............................] - ETA: 1:22:32 - loss: 11.9152
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_19_l_11.747794151306152.ckpt
 1045/36391 [..............................] - ETA: 1:18:44 - loss: 11.6741
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_19_l_11.657929420471191.ckpt
 2045/36391 [>.............................] - ETA: 1:16:36 - loss: 11.6689
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/checkpoints/w_tfidentity/ms1m_EfficientNetLite0_ArcHead_check/SGD/*SGD/e_19_l_11.644964218139648.ckpt
 3045/36391 [=>............................] - ETA: 1:14:15 - loss: 11.6

KeyboardInterrupt: 